In [20]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import re
from itertools import combinations
import urllib
from bs4 import BeautifulSoup as bs
import json
import datetime
from tqdm.notebook import tqdm
import pymysql
import traceback
import psycopg2
from psycopg2.extras import RealDictCursor
from appstoreconnect import Api
import pymssql
from dateutil.relativedelta import relativedelta
import pickle as pk

plt.style.use("dark_background")

path = "C:/Windows/Fonts/malgun.ttf"
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
mpl.rc("font", family=font_name)
mpl.rc("axes", unicode_minus=False)

pd.set_option("display.max_columns", None)

## 구글 애널리틱스
- https://ga-dev-tools.appspot.com/query-explorer/

### MAU
- metrics: "ga:users", "ga:pageviews"
- dimentions: "ga:yearMonth"

In [14]:
start = datetime.datetime(2017, 1, 1)
end = datetime.datetime(2021, 7, 31)
token = "ya29.a0ARrdaM8NE8YPegZVgbU81SfaeY3ih5yx2WpxAbTlTPSXT9fA_HKe6-0o-Pk6XmrhTPTDQVIb5WiXd1kS8FXGYvZ077BTJCleaiND8GNyR9znl685xFLlPaDvfkcA96GOK-Z4LmbO1d-MS-J8aSESqk7YRRxVRNw"

In [3]:
url = f"https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A206525354&start-date={start.strftime('%Y-%m-%d')}&end-date={end.strftime('%Y-%m-%d')}&metrics=ga%3Ausers%2Cga%3Apageviews&dimensions=ga%3AyearMonth&access_token={token}"
req = urllib.request.urlopen(url).read().decode("utf8")
js = json.loads(req)

mau = pd.DataFrame(js["rows"], columns=pd.DataFrame(js["columnHeaders"])["name"].str.split(":").str[1].tolist())

In [7]:
# mau.to_csv(f"D:/디지털혁신팀/태블로/마이호미/마이호미_FINAL/mau_{end.strftime('%Y-%m-%d')}.csv", encoding="euc-kr", index=False)
# mau.tail()

## mall.csv

In [6]:
# cols = ["주문번호", "아이디", "상품ID", "상품명", "수량", "상품금액", "주문일자", "모바일주문여부"]

# with open("D:/디지털혁신팀/태블로/mall_password.pkl", "rb") as f:
#      password = pk.load(f)
# conn = pymssql.connect(server="125.60.68.233", database="eiparkclub", user="myhomie", password=password, charset="utf8")
# cur = conn.cursor()

# sql = f"SELECT 주문번호, 임직원아이디, _상품ID, 상품명, 수량, 상품금액, 주문일자, _모바일주문여부 FROM vwAdjustDetail;"
# # sql = f"SELECT 임직원아이디 FROM vwAdjustDetail;"
# cur.execute(sql)

# data = pd.DataFrame(cur.fetchall(), columns=cols)
# # data = pd.DataFrame(cur.fetchall())
# data = data[data["아이디"].str.contains("MBR")]
# data = data.reset_index(drop=True)

In [8]:
# data.to_csv(f"mall_{end.strftime('%Y-%m-%d')}.csv", encoding="euc-kr", index=False)
# data.head()

# 다운로드(iOS)

In [1]:
# with open("D:/AuthKey_BB99VD4S7X.p8", "r") as f:
#     key_file = f.read()
# api = Api(key_id="BB99VD4S7X", key_file=key_file, issuer_id="252fd97c-ea6e-45b9-a795-693f38f8dbaf")
# api.download_sales_and_trends_reports(filters={"frequency": "DAILY", "reportSubType":"DETAILED", "reportType":"SALES", "vendorNumber": "88755846"})
# # , save_to="report.csv")

In [2]:
# views = pd.DataFrame()
# for i in tqdm(range(100)):
#     try:
#         start = end - datetime.timedelta(days=10*(i+1))
#         end = end - datetime.timedelta(days=10*i)
#         url = f"https://www.googleapis.com/analytics/v3/data/ga?ids=ga%3A206525354&start-date={start.strftime('%Y-%m-%d')}&end-date={end.strftime('%Y-%m-%d')}&metrics=ga%3Ausers%2Cga%3Apageviews&dimensions=ga%3ApageTitle%2Cga%3Adate&sort=-ga%3Adate&access_token={token}"
#         req = urllib.request.urlopen(url).read().decode("utf8")
#         js = json.loads(req)
        
#         df = pd.DataFrame(js["rows"], columns=pd.DataFrame(js["columnHeaders"])["name"].str.split(":").str[1].tolist())
#         views = pd.concat([views, df])
#     except:
#         break

## 개발사 데이터

## sys_code

In [24]:
# with open("D:/디지털혁신팀/태블로/signup_password.pkl", "rb") as f:
#      password = pk.load(f)
# conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
# cur = conn.cursor(cursor_factory=RealDictCursor)
# query = f"SELECT * FROM hdc_sys.sys_code;"
# cur.execute(query)
# result = cur.fetchall()

# sys_code = pd.DataFrame(result)

In [18]:
# sys_code.head()

## apt_mst.csv

In [19]:
with open("D:/디지털혁신팀/태블로/signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = f"SELECT * FROM hdc_apt.apt_mst;"
cur.execute(query)
result = cur.fetchall()

apt_mst = pd.DataFrame(result)

In [23]:
# apt_mst.to_csv(f"D:/디지털혁신팀/태블로/마이호미/마이호미_FINAL/apt_mst_{end.strftime('%Y-%m-%d')}.csv", encoding="euc-kr",  index=False)
# apt_mst.head()

## signup.csv

In [18]:
# 입주예정자
with open("D:/Github/Work/Tableau/signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = """SELECT A.*
          FROM (
        SELECT MBRMST.MBR_ID
        	 , MBRMST.LOGIN_ID
             , APTMST.APT_NAME
             , AUI.DNG_NO
             , AUI.HO_NO
             , AUI.SEP_NO
             , CONCAT(MBRMST.MBR_NAME,'(',MBRMST.LOGIN_ID, ')') AS MBR_NAME
             , MBRMST.MBR_NICKNAME
             , MBRMST.MBR_PHONE
             , MBRMST.MBR_SYS_TY
             , C1.CRD_NAME AS MBR_SYS_TY_NAME
             , MBRMST.MBR_ST
             , C2.CRD_NAME AS MBR_ST_NAME
             , MBRMST.REGIST_DT
             , HMMD.MBR_LAST_ACCESS_DT
             , MBRCERT.MBR_ACERT_ID
             , APTMST.APT_ID
             , C3.CRD_GROUP AS LOC1
             , APTMST.APT_ADDR_CODE
             , MBRMST.MBR_MEMO
             , TO_CHAR(MBRMST.MBR_LEAVING_EXPECTED_DATE, 'YYYY/MM/DD') AS MBR_LEAVING_EXPECTED_DATE 
             , AUI.UNIT_ID
             , MBRMST.MBR_ADMIN_YN
             , MBRMST.MBR_FAMILLY_TY
             , CASE WHEN MBRMST.MBR_DELETE_DATE IS NULL THEN ''
                    WHEN MBRMST.MBR_DELETE_DATE IS NOT NULL THEN TO_CHAR(MBRMST.MBR_DELETE_DATE,'YYYY/MM/DD')
                END MBR_DELETE_DATE 
          FROM HDC_MBR.MBR_MST MBRMST
	 LEFT JOIN HDC_MBR.MBR_DEVICE HMMD ON MBRMST.MBR_ID = HMMD.MBR_ID
     LEFT JOIN HDC_SYS.SYS_CODE C1 ON MBRMST.MBR_SYS_TY = C1.CRD
     LEFT JOIN HDC_SYS.SYS_CODE C2 ON MBRMST.MBR_ST = C2.CRD
	INNER JOIN HDC_MBR.MBR_CERT MBRCERT ON MBRMST.MBR_ID = MBRCERT.MBR_ID
	INNER JOIN HDC_MBR.MBR_APT_CERT_HIS ACH ON MBRCERT.MBR_ACERT_ID = ACH.APT_CERT_ID
	INNER JOIN HDC_APT.APT_MST APTMST ON ACH.APT_ID = APTMST.APT_ID
    INNER JOIN HDC_APT.APT_UNIT_INFO AUI ON ACH.APT_UNIT_ID = AUI.UNIT_ID
	 LEFT JOIN HDC_SYS.SYS_CODE C3 ON APTMST.APT_ADDR_CODE = C3.CRD) A
	WHERE 1=1
	AND A.MBR_ADMIN_YN = 'N'
	AND	A.MBR_SYS_TY = 'C000004001'
    ORDER BY A.REGIST_DT DESC"""
cur.execute(query)
result = cur.fetchall()
signup1 = pd.DataFrame(result)

In [ ]:
signup1 = signup[["mbr_id", "apt_name", "dng_no", "ho_no", "mbr_sys_ty_name", "mbr_st_name", "regist_dt", "mbr_delete_date"]]

In [21]:
signup1

,mbr_id,login_id,apt_name,dng_no,ho_no,sep_no,mbr_name,mbr_nickname,mbr_phone,mbr_sys_ty,mbr_sys_ty_name,mbr_st,mbr_st_name,regist_dt,mbr_last_access_dt,mbr_acert_id,apt_id,loc1,apt_addr_code,mbr_memo,mbr_leaving_expected_date,unit_id,mbr_admin_yn,mbr_familly_ty,mbr_delete_date
0,MBR0000027678,regina729,청주가경 아이파크 3단지,308,1306,None,권세진(regina729),별바람햇살,01048098689,C000004001,입주예정자,C000002001,정상,2021-08-03 13:11:46.715265,2021-08-03 13:16:47.718084,ACT0000055695,APT0000109911,C000015003,C005003001,None,None,UNI0000110919,N,C000017001,
1,MBR0000027677,kingflow,캐슬앤파밀리에시티1단지,119,102,None,이영기(kingflow),MJYK,01084622527,C000004001,입주예정자,C000002001,정상,2021-08-03 11:01:38.576269,NaT,ACT0000055693,APT0000079481,C000015002,C015002005,None,None,UNI0000080612,N,C000017001,
2,MBR0000027659,iroisie,캐슬앤파밀리에시티1단지,134,905,None,김민주(iroisie),민트캔디,01071714421,C000004001,입주예정자,C000002001,정상,2021-07-31 10:30:03.470228,2021-07-31 14:42:12.386150,ACT0000055647,APT0000079481,C000015002,C015002005,None,None,UNI0000081584,N,C000017002,
3,MBR0000027656,nina08,캐슬앤파밀리에시티1단지,102,405,None,김효진(nina08),별사랑h,01023198434,C000004001,입주예정자,C000002001,정상,2021-07-30 17:25:09.299165,2021-08-02 15:15:50.801517,ACT0000055645,APT0000079481,C000015002,C015002005,None,None,UNI0000079680,N,C000017001,
4,MBR0000027655,kds0814,캐슬앤파밀리에시티1단지,128,805,None,한우섭(kds0814),우섭,01067435925,C000004001,입주예정자,C000002001,정상,2021-07-30 16:07:11.764067,2021-07-31 18:28:23.453518,ACT0000055643,APT0000079481,C000015002,C015002005,None,None,UNI0000081205,N,C000017001,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16366,MBR0000000024,lilykk7,잠실 올림픽공원 아이파크,105,3002,None,김경희(lilykk7),블랙맘마,01025122090,C000004001,입주예정자,C000002001,정상,2019-10-24 09:55:46.869401,2020-04-16 19:53:43.346560,ACT0000000061,APT0000000921,C000015001,C015001001,None,None,UNI0000001459,N,C000017001,
16367,MBR0000000021,durkins,잠실 HDC 아파트,112,101,None,김영호(durkins),댄디가이,010226242511,C000004001,입주예정자,C000002005,탈퇴,2019-10-23 23:07:18.600666,NaT,ACT0000000055,APT0000001681,C000015001,C015001001,None,None,UNI0000001706,N,C000017001,
16368,MBR0000000020,APTADMIN,잠실 HDC 아파트,111,109,None,이광우(APTADMIN),마이호미,01088742159,C000004001,입주예정자,C000002005,탈퇴,2019-10-23 20:42:06.894896,NaT,ACT0000000053,APT0000001681,C000015001,C015001001,None,None,UNI0000001705,N,C000017001,
16369,MBR0000000019,seaka4444,잠실 HDC 아파트,111,110,None,이동균(seaka4444),skylook,01030217688,C000004001,입주예정자,C000002005,탈퇴,2019-10-23 19:37:18.981733,NaT,ACT0000000051,APT0000001681,C000015001,C015001001,None,None,UNI0000001745,N,C000017001,


In [10]:
# 입주예정자
with open("D:/Github/Work/Tableau/signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = """SELECT A.*
          FROM (
        SELECT MBRMST.MBR_ID
        	 , MBRMST.LOGIN_ID
             , APTMST.APT_NAME
             , AUI.DNG_NO
             , AUI.HO_NO
             , AUI.SEP_NO
             , CONCAT(MBRMST.MBR_NAME,'(',MBRMST.LOGIN_ID, ')') AS MBR_NAME
             , MBRMST.MBR_NICKNAME
             , MBRMST.MBR_PHONE
             , MBRMST.MBR_SYS_TY
             , C1.CRD_NAME AS MBR_SYS_TY_NAME
             , MBRMST.MBR_ST
             , C2.CRD_NAME AS MBR_ST_NAME
             , MBRMST.REGIST_DT
             , HMMD.MBR_LAST_ACCESS_DT
             , MBRCERT.MBR_ACERT_ID
             , APTMST.APT_ID
             , C3.CRD_GROUP AS LOC1
             , APTMST.APT_ADDR_CODE
             , MBRMST.MBR_MEMO
             , TO_CHAR(MBRMST.MBR_LEAVING_EXPECTED_DATE, 'YYYY/MM/DD') AS MBR_LEAVING_EXPECTED_DATE 
             , AUI.UNIT_ID
             , MBRMST.MBR_ADMIN_YN
             , MBRMST.MBR_FAMILLY_TY
             , CASE WHEN MBRMST.MBR_DELETE_DATE IS NULL THEN ''
                    WHEN MBRMST.MBR_DELETE_DATE IS NOT NULL THEN TO_CHAR(MBRMST.MBR_DELETE_DATE,'YYYY/MM/DD')
                END MBR_DELETE_DATE 
          FROM HDC_MBR.MBR_MST MBRMST
	 LEFT JOIN HDC_MBR.MBR_DEVICE HMMD ON MBRMST.MBR_ID = HMMD.MBR_ID
     LEFT JOIN HDC_SYS.SYS_CODE C1 ON MBRMST.MBR_SYS_TY = C1.CRD
     LEFT JOIN HDC_SYS.SYS_CODE C2 ON MBRMST.MBR_ST = C2.CRD
	INNER JOIN HDC_MBR.MBR_CERT MBRCERT ON MBRMST.MBR_ID = MBRCERT.MBR_ID
	INNER JOIN HDC_MBR.MBR_APT_CERT_HIS ACH ON MBRCERT.MBR_ACERT_ID = ACH.APT_CERT_ID
	INNER JOIN HDC_APT.APT_MST APTMST ON ACH.APT_ID = APTMST.APT_ID
    INNER JOIN HDC_APT.APT_UNIT_INFO AUI ON ACH.APT_UNIT_ID = AUI.UNIT_ID
	 LEFT JOIN HDC_SYS.SYS_CODE C3 ON APTMST.APT_ADDR_CODE = C3.CRD
	  UNION
        SELECT MBRMST.MBR_ID
             , MBRMST.LOGIN_ID
             , APTMST.APT_NAME
             , AUI.DNG_NO
             , AUI.HO_NO
             , AUI.SEP_NO
             , CONCAT(MBRMST.MBR_NAME,'(',MBRMST.LOGIN_ID, ')') AS MBR_NAME
             , MBRMST.MBR_NICKNAME
             , MBRMST.MBR_PHONE
             , MBRMST.MBR_SYS_TY
             , C1.CRD_NAME AS MBR_SYS_TY_NAME
             , MBRMST.MBR_ST
             , C2.CRD_NAME AS MBR_ST_NAME
             , MBRMST.REGIST_DT
             , HMMD.MBR_LAST_ACCESS_DT
             , MBRCERT.MBR_ACERT_ID
             , APTMST.APT_ID
             , C3.CRD_GROUP AS LOC1
             , APTMST.APT_ADDR_CODE
             , MBRMST.MBR_MEMO
             , TO_CHAR(MBRMST.MBR_LEAVING_EXPECTED_DATE, 'YYYY/MM/DD') AS MBR_LEAVING_EXPECTED_DATE 
             , AUI.UNIT_ID
             , MBRMST.MBR_ADMIN_YN
             , MBRMST.MBR_FAMILLY_TY
             , CASE WHEN MBRMST.MBR_DELETE_DATE IS NULL THEN ''
                    WHEN MBRMST.MBR_DELETE_DATE IS NOT NULL THEN TO_CHAR(MBRMST.MBR_DELETE_DATE,'YYYY/MM/DD')
                END MBR_DELETE_DATE
          FROM HDC_MBR.MBR_MST MBRMST
     LEFT JOIN HDC_MBR.MBR_DEVICE HMMD ON MBRMST.MBR_ID = HMMD.MBR_ID
     LEFT JOIN HDC_SYS.SYS_CODE C1 ON MBRMST.MBR_SYS_TY = C1.CRD
     LEFT JOIN HDC_SYS.SYS_CODE C2 ON MBRMST.MBR_ST = C2.CRD
	INNER JOIN HDC_MBR.MBR_GROUP_MBR MGM ON MBRMST.MBR_ID = MGM.MBR_ID AND MGM.GROUP_MBR_INVITE_ST != 'C000002009'
    INNER JOIN HDC_MBR.MBR_GROUP MG ON MGM.GRP_ID = MG.GRP_ID
 	 LEFT JOIN HDC_MBR.MBR_CERT MBRCERT ON MG.GRP_OWNER = MBRCERT.MBR_ID
 	 LEFT JOIN HDC_MBR.MBR_APT_CERT_HIS ACH ON MBRCERT.MBR_ACERT_ID = ACH.APT_CERT_ID
 	 LEFT JOIN HDC_APT.APT_MST APTMST ON ACH.APT_ID = APTMST.APT_ID
     LEFT JOIN HDC_APT.APT_UNIT_INFO AUI ON ACH.APT_UNIT_ID = AUI.UNIT_ID
	 LEFT JOIN HDC_SYS.SYS_CODE C3 ON APTMST.APT_ADDR_CODE = C3.CRD
	 ) A
	WHERE 1=1
	AND A.MBR_ADMIN_YN = 'N'
	AND	A.MBR_SYS_TY IN ('C000004003','C000004004')
    ORDER BY A.REGIST_DT DESC"""
cur.execute(query)
result = cur.fetchall()
b = pd.DataFrame(result)

In [11]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16371 entries, 0 to 16370
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   mbr_id                     16371 non-null  object        
 1   login_id                   16371 non-null  object        
 2   apt_name                   16371 non-null  object        
 3   dng_no                     16371 non-null  object        
 4   ho_no                      16371 non-null  object        
 5   sep_no                     133 non-null    object        
 6   mbr_name                   16371 non-null  object        
 7   mbr_nickname               16371 non-null  object        
 8   mbr_phone                  16371 non-null  object        
 9   mbr_sys_ty                 16371 non-null  object        
 10  mbr_sys_ty_name            16371 non-null  object        
 11  mbr_st                     16371 non-null  object        
 12  mbr_

In [14]:
b.sample(20)

,mbr_id,login_id,apt_name,dng_no,ho_no,sep_no,mbr_name,mbr_nickname,mbr_phone,mbr_sys_ty,...,mbr_acert_id,apt_id,loc1,apt_addr_code,mbr_memo,mbr_leaving_expected_date,unit_id,mbr_admin_yn,mbr_familly_ty,mbr_delete_date
4375,MBR0000011953,leekh0116,운정신도시 아이파크,105,2804,None,이광훈(leekh0116),꼭꼭숨어라,01053913112,C000004003,...,ACT0000022783,APT0000055389,C000015002,C015002004,None,None,UNI0000059740,N,C000017001,
6927,MBR0000000087,jdhome,잠실 올림픽공원 아이파크,102,1801,None,이재동(jdhome),라리스,01087789081,C000004003,...,ACT0000000192,APT0000000921,C000015001,C015001001,None,None,UNI0000001021,N,C000017001,
4840,MBR0000008190,lahasa83,산성역 포레스티아,110,2403,None,변경미(lahasa83),변경미,01046998682,C000004003,...,ACT0000015224,APT0000043153,C000015002,C015002003,None,None,UNI0000044097,N,C000017002,
4490,MBR0000010872,thdud4776,목동센트럴아이파크위브,207,1401,None,박소영(thdud4776),풀봉이,01062994776,C000004003,...,ACT0000020591,APT0000007902,C000015001,C015001003,None,None,UNI0000035873,N,C000017001,
4980,MBR0000007212,ju38210,산성역 포레스티아,105,1603,None,김명희(ju38210),민준파파,01086197460,C000004003,...,ACT0000013318,APT0000043153,C000015002,C015002003,None,None,UNI0000043664,N,C000017001,
3427,MBR0000020841,kcj711,별내역 아이파크 스위트,103,2906,,김철주(kcj711),병포,01045180737,C000004003,...,ACT0000040013,APT0000092369,C000015002,C015002007,None,None,UNI0000093346,N,C000017001,
2939,MBR0000021656,jungij12,병점역 아이파크 캐슬,127,603,None,정일중(jungij12),정일중,01030078917,C000004003,...,ACT0000041585,APT0000094182,C000015002,C015002002,None,None,UNI0000100099,N,C000017001,
1592,MBR0000023467,ju704,병점역 아이파크 캐슬,127,1601,None,주성태(ju704),주꿈이,01041665681,C000004003,...,ACT0000045260,APT0000094182,C000015002,C015002002,None,None,UNI0000100061,N,C000017001,
5852,MBR0000002949,beautiful66,잠실 올림픽공원 아이파크,103,1706,None,정금숙(beautiful66),ggg,01042204546,C000004004,...,ACT0000005093,APT0000000921,C000015001,C015001001,None,None,UNI0000001252,N,C000017002,2021/07/27
1122,MBR0000024223,eun1008,대구 복현 아이파크,103,807,None,강은숙(eun1008),은실,01087516567,C000004003,...,ACT0000046743,APT0000102074,C000015004,C005004001,None,None,UNI0000104871,N,C000017001,


In [12]:
b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6962 entries, 0 to 6961
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   mbr_id                     6962 non-null   object        
 1   login_id                   6962 non-null   object        
 2   apt_name                   6962 non-null   object        
 3   dng_no                     6962 non-null   object        
 4   ho_no                      6962 non-null   object        
 5   sep_no                     148 non-null    object        
 6   mbr_name                   6962 non-null   object        
 7   mbr_nickname               6962 non-null   object        
 8   mbr_phone                  6962 non-null   object        
 9   mbr_sys_ty                 6962 non-null   object        
 10  mbr_sys_ty_name            6962 non-null   object        
 11  mbr_st                     6962 non-null   object        
 12  mbr_st

In [15]:
with open("D:/Github/Work/Tableau/signup_password.pkl", "rb") as f:
     password = pk.load(f)
conn = psycopg2.connect(host="postgresql-production1.cluster-ro-cxb5zwyq1mu5.ap-northeast-2.rds.amazonaws.com", dbname="hdc", user="postgres", password=password)
cur = conn.cursor(cursor_factory=RealDictCursor)
query = f"SELECT hdc_mbr.mbr_mst.mbr_id, \
CASE hdc_mbr.mbr_mst.mbr_sys_ty  \
WHEN 'C000004001' THEN '입주예정자' \
WHEN 'C000004003' THEN '입주자(대표)' \
WHEN 'C000004004' THEN '입주자(세대원)' \
END AS mbr_sys_ty, \
CASE hdc_mbr.mbr_mst.mbr_sex \
WHEN '0' THEN '남자' \
WHEN '1' THEN '여자' \
END AS mbr_sex, \
hdc_mbr.mbr_mst.mbr_birth, hdc_mbr.mbr_mst.regist_dt, hdc_mbr.mbr_mst.mbr_delete_date, \
CASE hdc_mbr.mbr_mst.mbr_st \
WHEN 'C000002001' THEN '정상' \
WHEN 'C000002005' THEN' 탈퇴' \
WHEN 'C000002007' THEN '퇴거' \
WHEN 'C000002008' THEN '휴면상태' \
END AS mbr_st, \
hdc_apt.apt_unit_mbr.mbr_id, \
hdc_apt.apt_unit_info.dng_no, hdc_apt.apt_unit_info.ho_no, \
hdc_apt.apt_mst.apt_name  \
FROM hdc_mbr.mbr_mst  \
LEFT OUTER JOIN hdc_apt.apt_unit_mbr  \
LEFT OUTER JOIN hdc_apt.apt_unit_info  \
LEFT OUTER JOIN hdc_apt.apt_mst  \
ON hdc_apt.apt_unit_info.apt_id = hdc_apt.apt_mst.apt_id  \
ON hdc_apt.apt_unit_mbr.unit_id = hdc_apt.apt_unit_info.unit_id  \
ON hdc_mbr.mbr_mst.mbr_id = hdc_apt.apt_unit_mbr.mbr_id;"
cur.execute(query)
result = cur.fetchall()
signup = pd.DataFrame(result)

In [17]:
signup.to_csv("signup.csv", encoding="euc-kr")

In [12]:
# signup = signup[signup["apt_name"].notna()]
# signup["mbr_sys_ty"] = signup["mbr_sys_ty"].map({"C000004001":"입주예정자", "C000004003":"입주자(대표)", "C000004004":"입주자(세대원)"})
# signup["mbr_sex"] = signup["mbr_sex"].map({"0":"남자", "1":"여자"})
# signup["mbr_st"] = signup["mbr_st"].map({"C000002001":"정상", "C000002005":"탈퇴", "C000002007":"퇴거", "C000002008":"휴면상태"})
# signup = signup.rename({"mbr_id":"아이디", "apt_name":"아파트 명", "dng_no":"동", "ho_no":"호", "mbr_sys_ty":"회원유형", "mbr_st":"회원상태", "mbr_sex":"성별", "mbr_birth":"생년월일", "regist_dt":"가입일", "mbr_delete_date":"탈퇴일"}, axis=1)
# signup["성별/생년월일"] = signup["성별"] + "/" + signup["생년월일"]
# signup = signup[["아이디", "아파트 명", "동", "호", "회원유형", "회원상태", "성별/생년월일", "가입일", "탈퇴일"]]

In [15]:
signup.to_csv(f"D:/Github/Work/Tableau/Myhomie/signup_{end.strftime('%Y-%m-%d')}.csv", encoding="euc-kr")